# Unilever

### Automatización aplicación pagos

In [24]:
# 1. Imports y configuración

import pandas as pd
import numpy as np

from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import numbers

from openpyxl.styles import Alignment # Formatear columnas


In [50]:
remittance = pd.read_excel(
    "Remittance.xlsx",
    skiprows=25,
    nrows=48,
    usecols=["DOC", "No. Doc", "Total a Pagar"]
).dropna(subset=["DOC"])

In [51]:
# Tranformo Remittence
## Cada cliente manda la informacion en Excel con distintos formatos (difrente indice) y a la vez para cada cliente la cantidad de filas va a variar

remittance = remittance.rename(columns={
    "DOC": "Tipo de Documento",
    "No. Doc": "Referencia / Factura",
    "Total a Pagar": "Importe de factura"
})

remittance["Referencia / Factura"] = remittance["Referencia / Factura"].str[1:-3]

remittance["Tipo de Documento"] = remittance["Tipo de Documento"].replace("Factura Comercial", "Factura")
remittance["Tipo de Documento"] = remittance["Tipo de Documento"].replace("Nota Crédito", "Descuentos no asociados a FC")

remittance["Importe de factura"] = pd.to_numeric(remittance["Importe de factura"], errors="coerce").round(2)

In [52]:
# Creamos columnas vacías (si no existen)
if "Descuento" not in remittance.columns:
    remittance["Descuento"] = ""

if "Motivo del descuento" not in remittance.columns:
    remittance["Motivo del descuento"] = ""

# Definimos condiciones y valores
conds = [
    (remittance["Referencia / Factura"].str.startswith("PMP", na=False)) & (remittance["Importe de factura"] < 0),
    remittance["Referencia / Factura"].str.startswith("0085489", na=False),
    remittance["Referencia / Factura"].str.startswith("463", na=False),
    remittance["Referencia / Factura"].str.startswith("9801649", na=False)
]

descuentos = ["RECHAZO", "DESCUENTO", "AVERIA", "FACT PROVEEDOR"]
motivos = ["551", "987", "522", "CSB"]

# Asignamos Descuento y Motivo del descuento
remittance["Descuento"] = np.select(conds, descuentos, default=remittance["Descuento"])
remittance["Motivo del descuento"] = np.select(conds, motivos, default=remittance["Motivo del descuento"])


In [53]:
# Agrego dato de Comentario
remittance["Comentarios"] = np.where(
    remittance["Tipo de Documento"] != "Factura",
    remittance["Descuento"].fillna("") + " " + remittance["Referencia / Factura"].fillna(""),
    ""
)

In [55]:
# Lectura de cartera FBL5N

# Leer solo las columnas necesarias
FBL5N = pd.read_excel(
    "FBL5N Olimpica Ago 18.xlsx",
    sheet_name="FBL5N Olimpica Ago 18",
    usecols=["Type", "Reference", "    Amt in loc.cur."]
)
# Agregar a la importacion de cartera FBL5N que traiga info cuando: Type = "YE" y RCd = "NRO"


# Filtrar directamente Type == RV
FBL5N = FBL5N[FBL5N["Type"] == "RV"]

# Renombrar columnas
FBL5N = FBL5N.rename(columns={
    "Reference": "Referencia / Factura",
    "    Amt in loc.cur.": "importe_FBL5N"
}).reset_index(drop=True)


In [56]:
# Transformo FBL5N
## Dato FBL5N (se puede hacer antes)
# Paso 1: eliminar puntos de miles y reemplazar coma por punto decimal
FBL5N["importe_FBL5N"] = FBL5N["importe_FBL5N"].str.replace(".", "", regex=False)  # quita separador de miles
FBL5N["importe_FBL5N"] = FBL5N["importe_FBL5N"].str.replace(",", ".", regex=False)  # convierte decimal a punto
# Paso 2: convertir a float
FBL5N["importe_FBL5N"] = pd.to_numeric(FBL5N["importe_FBL5N"], errors="coerce")

In [84]:
# Cruzo Remittance y FBL5N por "Referencia / Factura"
merged_df = pd.merge(
    remittance,
    FBL5N,
    on="Referencia / Factura",   # mismo nombre en ambos
    how="left"                   # mantiene todas las filas de remittance
)


In [85]:
# Creo Diferencia entre Importe de factura (Remittence) y importe_FBL5
merged_df["Diferencia"] = merged_df["importe_FBL5N"] - merged_df["Importe de factura"]

In [86]:
# Inicializamos la columna como NaN
merged_df["Diferencia"] = pd.NA

# Calculamos la diferencia solo para facturas
merged_df.loc[merged_df["Tipo de Documento"] == "Factura", "Diferencia"] = (
     merged_df["importe_FBL5N"] - merged_df["Importe de factura"]
)

In [87]:
# Filtramos filas donde Diferencia no es NA y distinta de 0 (esto cambia con respecto al Template, menos dos registros)
diferencias = merged_df[merged_df["Diferencia"].notna() & (merged_df["Diferencia"] != 0)].copy()

# Creamos las nuevas filas según tus reglas
registros_diferencias_entre_remittence_cartera = pd.DataFrame({
    "Tipo de Documento": "Descuentos no asociados a FC",
    "Referencia / Factura": diferencias["Referencia / Factura"],
    "Importe de factura": diferencias["Diferencia"],
    "Pago Neto": "",  # opcional
    "Descuento": diferencias["Diferencia"].apply(
        lambda x: "MENORES VALORES" if -2000 < x < 2000 else "A definir"
    ),
    "Motivo del descuento": diferencias["Diferencia"].apply(
        lambda x: "WOB" if x < 0 else "384"
    )
})

# Agregamos la columna Comentarios como concatenación de Descuento + Tipo de Documento
registros_diferencias_entre_remittence_cartera["Comentarios"] = (
    registros_diferencias_entre_remittence_cartera["Descuento"] + " " +
    registros_diferencias_entre_remittence_cartera["Referencia / Factura"]
)

# Concatenamos las nuevas filas al DataFrame original
merged_df = pd.concat([merged_df, registros_diferencias_entre_remittence_cartera], ignore_index=True)


In [88]:
merged_df["Pago Neto"] = merged_df["Importe de factura"]
merged_df["Referencia / Factura"] = merged_df["Referencia / Factura"].astype(str)

In [89]:
# Limpio las columnas con las que me voy a quedar en Template

columnas_finales = [
    "Tipo de Documento",
    "Referencia / Factura",
    "Importe de factura",
    "Descuento",
    "Motivo del descuento",
    "Pago Neto",
    "Comentarios"
]

merged_df = merged_df[columnas_finales]

In [96]:
merged_df.head(100)

,Tipo de Documento,Referencia / Factura,Importe de factura,Descuento,Motivo del descuento,Pago Neto,Comentarios
0,Factura,PMP1243434,165989699,,,165989699,
1,Factura,PMP1243435,29856410,,,29856410,
2,Factura,PMP1243436,12528130,,,12528130,
3,Factura,PMP1243437,83124,,,83124,
4,Factura,PMP1243438,53514044,,,53514044,
5,Factura,PMP1243439,757032037,,,757032037,
6,Factura,PMP1243546,122759901,,,122759901,
7,Factura,PMP1243587,132622525,,,132622525,
8,Factura,PMP1243639,343873745,,,343873745,
9,Descuentos no asociados a FC,0085489002,-106616,DESCUENTO,987,-106616,DESCUENTO 0085489002


In [17]:
# Exportacion de archivo

In [91]:
ruta_salida = "merged_template.xlsx"

# Exportamos con pandas, indicando hoja y posición inicial
merged_df.to_excel(
    ruta_salida,
    index=False,
    sheet_name="Template",
    startrow=17,
    startcol=2
)

In [92]:
# Abrimos el archivo para aplicar formatos
wb = load_workbook(ruta_salida)
ws = wb["Template"]

In [99]:
# Formato del Template:
# Columnas numéricas
num_cols = ["Importe de factura", "Pago Neto"]

for col in num_cols:
    col_idx = merged_df.columns.get_loc(col) + 3  # startcol=2 → columna C = 3 en openpyxl
    for row in range(18, 18 + len(merged_df) + 1):  # largo de df +1
        ws.cell(row=row, column=col_idx).number_format = '#,##0.00'
        
# Columnas de texto
## Columnas de texto formateadas y centradas (excepto 'Comentarios')
str_cols = ["Tipo de Documento", "Referencia / Factura", "Descuento", "Motivo del descuento", "Comentarios"]

for col in str_cols:
    col_idx = merged_df.columns.get_loc(col) + 3
    for row in range(18, 18 + len(merged_df) + 1):  # largo de df +1
        cell = ws.cell(row=row, column=col_idx)
        cell.number_format = '@'  # formato texto
        if col != "Comentarios":
            cell.alignment = Alignment(horizontal="center", vertical="center")


In [100]:
# Guardar cambios en el archivo Excel
wb.save(ruta_salida)
print(f"Archivo exportado correctamente con formato: {ruta_salida}")

Archivo exportado correctamente con formato: merged_template.xlsx
